In [25]:
import ee 

In [26]:
ee.Authenticate() 
ee.Initialize() 

In [42]:
class Geoindexity:
    """
    The core geoindexity time series object. 
    ...

    Attributes
    ----------
    aoi : list
        [xmin, ymin, xmax, ymax]
    startdate : str
        'yyyy-mm-dd'
    enddate : str
        'yyyy-mm-dd'
    product: str 
        GEE data set: 'S1', 'S2_TOA', 'S2_SR' 
    cloudcover: int 
        percentage cloudcover threshold for series

    Methods
    -------
    fetch_data(self):
        Fetches GEE Image collection based on the following time_series attributes:
        product, aoi, startdate, enddate, cloudcover

    """
    # Will be changed accordingly to the config file parameters

    def __init__(self, aoi, startdate, enddate, product, cloudcover):
        data_dict = {
            'S2_TOA': 'COPERNICUS/S2_HARMONIZED',
            'Sentinel': 'COPERNICUS/S2_SR_HARMONIZED',
            'Landsat': 'LANDSAT/LC08/C02/T1_L2'}
        
        self.aoi = ee.Geometry.Rectangle(aoi)
        self.startdate = startdate
        self.enddate = enddate
        self.product = data_dict[product] 
        self.cloudcover = cloudcover
        self.collection = ee.ImageCollection(self.product).filterBounds(self.aoi).filterDate(self.startdate, self.enddate).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', self.cloudcover))
    def __len__(self): 
        if self.collection != None:
            return self.collection.size().getInfo()
        else: 
            return 0

    def add_NDVI(self, image):
        if self.product == 'COPERNICUS/S2_SR_HARMONIZED':
            ndvi = image.expression( 
                '((nir - red)/(nir + red))',
                {'nir': image.select('B4'),
                 'red': image.select('B8')}
            ).rename('NDVI')
        elif self.product == 'LANDSAT/LC08/C02/T1_L2': 
            ndvi = image.expression( 
                '((nir - red)/(nir + red))',
                {'nir': image.select('SR_B5'),
                 'red': image.select('SR_B4')}
            ).rename('NDVI')      
            
        return image.addBands(ndvi)


    def add_EVI(self, img): 
        if self.product == 'COPERNICUS/S2_SR_HARMONIZED':
              evi = img.expression(
                '2.5 * ((nir – red) / (nir + 6 * red – 7.5 * blue + 1))',
                {'nir': img.select('B8'),
                 'red': img.select('B4'),
                 'blue': img.select('B2')}
              ).rename('EVI')

        elif self.product == 'LANDSAT/LC08/C02/T1_L2':
            evi = img.expression(
                '(2.5 * (nir–red) / (nir + 6 * red – 7.5 * blue + 1)',
                {'nir': img.select('SR_B5'),
                 'red': img.select('SR_B4'),
                 'blue': img.select('SR_B2')}
            ).rename('EVI')
        
        return img.addBands(evi)
        
    def ndvi_collection(self):
       self.collection = self.collection.map(self.add_NDVI) 

    def evi_collection(self):
        self.collection = self.collection.map(self.add_EVI) 


In [43]:
aoi = [-2.9,
       47.48,
       -2.6,
       47.63
      ]


x = Geoindexity(aoi,
            startdate='2023-06-01',
            enddate='2023-08-30', 
            product= 'Sentinel', 
            cloudcover=10)

In [44]:
x.ndvi_collection()

In [45]:
x.evi_collection()

In [46]:
x.collection.getInfo()

EEException: Image.parseExpression: Expression parse error at character 12:
'2.5 * ((nir – red) / (nir + 6 * red – 7.5 * blue + 1))'
             ^.